In [15]:
import numpy as np 
import pandas as pd
import seaborn as sns
import os
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

Input_Matrix = pd.read_excel('/home/sai/Pyt-Ex/Data/as of 2020 Historical Data.xlsx', sheet_name='Input Matrix')

Target_Matrix = pd.read_excel('/home/sai/Pyt-Ex/Data/as of 2020 Historical Data.xlsx', sheet_name='Target Matrix')


In [16]:
input_columns = ['Year', 'Storm', 'Population_Affected', 'Pressure(mbar)', 'Wind_Speed(mph)', 'Storm_Surge(ft)', 'Precip(inches)', 'First_Latitude', 'First_Longtitude', 'Second_Latitude', 'Second_Longtitude', 'Third_Latitude', 'Third_Longtitude', 'Fourth_Latitude', 'Fourth_Longtitude']

Input_Matrix.set_axis(input_columns, axis=1, inplace=True)

Input_Matrix.drop([0], inplace = True)

Input_Matrix.drop(['Year', 'Storm'], axis=1, inplace=True)

Input_Matrix

,Population_Affected,Pressure(mbar),Wind_Speed(mph),Storm_Surge(ft),Precip(inches),First_Latitude,First_Longtitude,Second_Latitude,Second_Longtitude,Third_Latitude,Third_Longtitude,Fourth_Latitude,Fourth_Longtitude
1,710000,1005,51.75,1.32,15,32.9,79.7,0,0,0,0,0,0
2,7200000,990,51.75,6.2,13.65,29.3,89.8,0,0,0,0,0,0
3,22500000,999,51.75,2.67,6.97,39.4,74.4,0,0,0,0,0,0
4,2000000,973,92,6.24,15.49,26.8,97.3,0,0,0,0,0,0
5,56300000,986,92,6.3,8.85,33.9,78.5,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1600000,987,80.5,5.3,16.38,30.1,85.7,0,0,0,0,0,0
93,17000000,990,51.75,5.1,11.38,28.2,96.9,0,0,0,0,0,0
94,11200000,964,103.5,9,38.46,24.5,81.8,30.4,88.9,0,0,0,0
95,2200000,1000,40.25,0,1,29.1,90.9,0,0,0,0,0,0


In [17]:
Input_Matrix.isnull().sum()

Population_Affected    0
Pressure(mbar)         0
Wind_Speed(mph)        0
Storm_Surge(ft)        0
Precip(inches)         0
First_Latitude         0
First_Longtitude       0
Second_Latitude        0
Second_Longtitude      0
Third_Latitude         0
Third_Longtitude       0
Fourth_Latitude        0
Fourth_Longtitude      0
dtype: int64

In [18]:
Target_Matrix

,Unnamed: 0,Unnamed: 1,RY Damage,2016 USD,Impact Level,IL 0,IL 1,IL 2,IL 3,IL 4,IL 5,Unnamed: 11,Check,Unnamed: 13
0,2020,Bertha,NaN,0.000000e+00,0,1,0,0,0,0,0,NaN,0,Y
1,NaN,Cristobal,3.100000e+08,2.666000e+08,2,0,0,1,0,0,0,NaN,2,Y
2,NaN,Fay,2.200000e+08,1.892000e+08,2,0,0,1,0,0,0,NaN,2,Y
3,NaN,Hanna,1.100000e+09,9.460000e+08,2,0,0,1,0,0,0,NaN,2,Y
4,NaN,Isaias,4.800000e+09,4.128000e+09,3,0,0,0,1,0,0,NaN,3,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,NaN,H. Earl,7.900000e+07,1.374509e+08,2,0,0,1,0,0,0,NaN,2,Y
92,NaN,TS. Frances,5.000000e+08,8.699422e+08,2,0,0,1,0,0,0,NaN,2,Y
93,NaN,H. Georges,5.900000e+09,1.026532e+10,4,0,0,0,0,1,0,NaN,4,Y
94,NaN,TS. Hermine,0.000000e+00,0.000000e+00,0,1,0,0,0,0,0,NaN,0,Y
